In [1]:
from transformers import AutoModel,AutoTokenizer , Trainer,TrainingArguments,BitsAndBytesConfig
from peft import get_peft_model,LoraConfig
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
import deepspeed
import os
cache_dir='/proj/ciptmp/ix05ogym/.cache/'


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No ROCm runtime is found, using ROCM_HOME='/usr'


[2024-06-11 13:35:14,136] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


In [2]:
train_dataset = datasets.load_dataset('McGill-NLP/WebLINX',split=['train[:100]'],cache_dir=cache_dir)[0]
train_dataset[0]


{'demo': 'cptbbef',
 'turn': 4,
 'action': 'click(uid=None)',
 'action_history': 'say(speaker="instructor", utterance="Hello") say(speaker="navigator", utterance="Hi")</s><s>[INST] say(speaker="instructor", utterance="Open momondo.in and login with google using the below details: \\n\\t\\n\\tId: webtasks.navigator@gmail.com \\n\\tPassword: KEG24qweUHij%^") load(url="https://www.momondo.in/")</s><s>[INST]',
 'utterances': 'N o   i n s t r u c t o r   u t t e r a n c e ;',
 'candidates': "(uid = 9f2c37b3-a223-4f07) [[tag]] body [[xpath]] /html/body [[text]]   [[bbox]] x=0 y=0 width=1519.2 height=2919.5 [[attributes]] id='c-ehw' data-webtasks-id='9f2c37b3-a223-4f07' class='keel-mom kl kl-override FlightsSearchBrandsMomon...st wide a11y-focus-outlines wide-fd en_IN horizon' [[children]]",
 'clean_html': '(html(head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#" ) (body class="keel-mom kl kl-override FlightsSearchBrandsMomondo react react-st wide a11y-focus-outlines wide-fd en_IN h

In [3]:
model_name = "PrunaAI/McGill-NLP-Llama-3-8B-Web-HQQ-4bit-smashed"
#config = BitsAndBytesConfig(
    #load_in_4bit=True,

    #load_in_8bit=True,
    
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_compute_dtype=torch.bfloat16,
    #)

"""
model = AutoModel.from_pretrained(model_name,
                          #quantization_config=GPTQConfig(bits=4, disable_exllama=False),
                          #load_in_8bit=True,
                          cache_dir=cache_dir)


print(model)
"""
#https://huggingface.co/docs/transformers/main/en/quantization/hqq
try:
 model_hqq = HQQModelForCausalLM.from_quantized(model_name, device_map='auto',cache_dir=cache_dir)#.to(torch.bfloat16) #.half()
except: 
 model_hqq = AutoHQQHFModel.from_quantized(model_name,cache_dir=cache_dir)
 
 

tokenizer = AutoTokenizer.from_pretrained("McGill-NLP/Llama-3-8B-Web")


100%|██████████| 225/225 [00:00<00:00, 20331.75it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
#model = model.eval()
#input_ids = tokenizer("create an html apply button?", return_tensors='pt').to('cuda')["input_ids"]
#outputs = model.generate(input_ids, max_new_tokens=216)
#tokenizer.decode(outputs[0])

In [5]:
print(model_hqq)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): HQQLinear(in_features=4096, out_features=4096, bias=False)
          (k_proj): HQQLinear(in_features=4096, out_features=1024, bias=False)
          (v_proj): HQQLinear(in_features=4096, out_features=1024, bias=False)
          (o_proj): HQQLinear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): HQQLinear(in_features=4096, out_features=14336, bias=False)
          (up_proj): HQQLinear(in_features=4096, out_features=14336, bias=False)
          (down_proj): HQQLinear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSN

In [6]:
snapshot_download("McGill-NLP/WebLINX", repo_type="dataset", allow_patterns="templates/*",cache_dir=cache_dir)
valid = datasets.load_dataset("McGill-NLP/WebLINX", split="validation[:5]",cache_dir=cache_dir)
template = """<s>[INST] <<SYS>>
{clean_html}
You will find above the HTML elements that are available for the current webpage.
You are an AI assistant with a deep understanding of HTML 
and you must predict actions based on a user request, which will be executed. 
Use one of the following, replacing [] with an appropriate value: change(value=[str], uid=[str]) ; click(uid=[str]) ; load(url=[str]) ; say(speaker="navigator", utterance=[str]) ; scroll(x=[int], y=[int]) ; 
submit(uid=[str]) ; text_input(text=[str], uid=[str]) ;
The user's first and last 4 utterances are:
{utterances} ;
Viewport size: {viewport}
Only the last 5 turns are provided.
Here are the top candidates for this turn:
{candidates}

<</SYS>>[/INST]
{action_history}
Please select the best action using the correct format, do not provide any other information or explanation.
[/INST]"""


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 2322.43it/s]


In [7]:
state = template.format(**valid[0])
print(state)

<s>[INST] <<SYS>>
(html(body(div class="container"(div class="row"(div class="col hdr-r justify-...flex align-items-center"(div class="hdr-categories-container"(a class="rc-link" onclick="if (!window.__cfRLUn... false; toggleFlyout()" data-webtasks-id="76978...85e-4cf1"(span class="text" data-webtasks-id="6c7fe1...640-4dce"EXPLORE)(i class="fa ency-down" data-webtasks-id="e7b787...5ae-48a5"))(div class="rc-flyout"))))) (div (div class="dialog-off-canvas-main-canvas"(div class="homepage"(div style="background-image: url('/sites...01_3.png');" class="ency-loaded"(div class="ency-loaded mask-hero")(h4 data-webtasks-id="1ea51e...fcd-4e30"The World’s #1 Online Encyclopedia)(div class="clear-both hero"(div class="ency-hero-search"(form action="https://www.encyclopedia.com/gsearch" method="get" data-webtasks-id="c7fbc11c...49-4ab2"(div class="js-form-item form-...-keys form-no-label" data-webtasks-id="8d8afc8...7-477a" (span class="field-preffix" (input class="button js-form-submit form-submi

In [12]:
"""
{
  "train_batch_size": 8,
  "gradient_accumulation_steps": 1,
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 0.00015
    }
  },
  "fp16": {
    "enabled": true
  },
  "zero_optimization": true
}
"""
#os.environ['DS_SKIP_CUDA_CHECK'] = '1'

"""
  "optimizer": {
      "type": "AdamW",
      "params": {
        "lr": 2e-5,
        "weight_decay": 0.0,
        "bias_correction": True
      }
    },

"""
ds_config = {
    "train_batch_size": 1,
    "gradient_accumulation_steps": 1,
    "gradient_clipping": 1.0,
      "optimizer": {
      "type": "AdamW",
      "params": {
        "lr": 2e-5,
        "weight_decay": 0.0,
        "bias_correction": True
      }
    },
  
   """ "quantize_training": {
      "enabled": True,
      "quantize_verbose": True,
      "quantizer_kernel": True,
      "quantize-algo": {
        "q_type": "symmetric"
      },
      "quantize_bits": {
        "start_bits": 4,
        "target_bits": 2
      },
      "quantize_schedule": {
        "quantize_period": 400,
        "schedule_offset": 0
      },
      #"quantize_groups": 8,
    },"""
  
    
    "bf16": {
        "enabled": True,
        #"initial_scale_power": 16,

    },
    "zero_optimization": {
        "stage": 3,
        "offload_param": {
            "device": "cpu",
            "pin_memory": True
        },
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
    },
    "activation_checkpointing": {
        "partition_activations": True,
        "cpu_checkpointing": True
    },
}
os.environ["PATH"] = '/usr/bin/g++'
os.environ["DEFAULT_TORCH_EXTENSION_PATH "] = '/proj/ciptmp/ix05ogym/tmp'
# Move the optimizer instance to the GPU
#optimizer = optimizer.to("cuda")


model, optimizer, _, _ = deepspeed.initialize(
    model=model_hqq,
    model_parameters=model_hqq.parameters(),
    #optimizer=deepspeed.ops.adam.DeepSpeedCPUAdam,
    config=ds_config
    ,
)

[2024-06-11 13:37:19,495] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.14.2, git-hash=unknown, git-branch=unknown
[2024-06-11 13:37:19,500] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False


Using /home/cip/ce/ix05ogym/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
No modifications detected for re-loaded extension module cpu_adam, skipping build step...
Loading extension module cpu_adam...


ImportError: /home/cip/ce/ix05ogym/.cache/torch_extensions/py311_cu118/cpu_adam/cpu_adam.so: cannot open shared object file: No such file or directory

In [ ]:
tokenized_state = tokenizer(state,return_tensors='pt',truncation=True,max_length=2048).to('cuda')#.to(torch.bfloat16)
print(tokenized_state)
#model = model.eval()
with torch.cuda.amp.autocast():#torch.no_grad():
    o=model(**tokenized_state)
    #del tokenized_state
    #torch.cuda.empty_cache()
    
o
#print("Action:", out['generated_text'])

In [ ]:
y =o.logits.cpu()

In [ ]:
m = y.argmax(2)
print(m.shape)
pp=tokenizer.decode(m[0])



In [ ]:
print(pp)